# Cambridge ML Commando Course
## Session 2.1 - Classification with Logistic Regression

In [ ]:
%pylab inline

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import sklearn
import IPython
import platform

from sklearn import preprocessing

print ('Python version:', platform.python_version())
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sklearn.__version__)
print ('matplotlib version:', matplotlib.__version__)

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

print(iris.target)
print(iris.target_names)

print("Data shape={}, type={}, labels={}\n".format(X_iris.shape, type(X_iris), iris.feature_names))
print("Target shape={}, type={}, labels={}".format(y_iris.shape, type(y_iris), y_labels))

In [ ]:
from sklearn.preprocessing import StandardScaler
manual_split=True
remove_setosa = True

if remove_setosa:
    indices_no_setosa = (iris.target != 0) #[TRUE for non-setosa else FALSE]
    X = iris.data[indices_no_setosa]
    y = iris.target[indices_no_setosa]
else:
    X = iris.data
    y = iris.target
    
if manual_split:
    numpy.random.seed(42)
    test_split = 0.33
    test_n = int(test_split*len(X))
    test_indices = numpy.random.randint(0,len(X), size=test_n)
    train_indices = list(set(range(len(X))) - set(test_indices))

    X_train  = X[:,[2,3]][train_indices]    
    y_train = y[train_indices]
    X_test = X[:,[2,3]][test_indices] 
    y_test = y[test_indices]
else:
    # convenience class to do the above...
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X[:,[2,3]], y, test_size=0.33, random_state=666)
    
print(X_train.shape)
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_test = X_scaler.transform(X_test)

ax = plt.gca()
ax.scatter(X_train[:,0], X_train[:,1], c=y_train, alpha=0.5, cmap=plt.cm.Paired)
ax.scatter(X_test[:,0], X_test[:,1], marker="s", s=50, c=y_test, edgecolor="k", cmap=plt.cm.Paired) #it's sneaky to look at your test data!
# plt.legend()
plt.xlabel('Petal length')
plt.ylabel('Petal width')
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(estr, X, y, K):
    # create a k-fold croos validation iterator of k=5 folds
    print("underway...")
    k_cv = KFold(n_splits=K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(estr, X, y, cv=k_cv)
    print (scores)
    print (("Mean score: {0:.3f} (+/-{1:.3f})").format(
        np.mean(scores), sem(scores)))

In [ ]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()#multi_class="ovr", solver="liblinear")#(multi_class="multinomial")
# from sklearn import svm
# reg = svm.SVC(kernel="poly")
reg.fit(X_train, y_train)

In [ ]:
evaluate_cross_validation(reg, X_train, y_train, 5)
y_hats_tr = reg.predict(X_train)
y_hats_proba = reg.predict_proba(X_train)
print("y yh[class probabilities]:")
for y,yh,yhp in zip(y_train, y_hats_tr, y_hats_proba):
    print(y, yh, yhp)


In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X_train[:, 0].min() - .5, X_train[:, 0].max() + .5
y_min, y_max = X_train[:, 1].min() - .5, X_train[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
# Z = reg_allf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = reg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(6, 6))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot the training and test points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker="o", s=20, edgecolors='k', cmap=plt.cm.Paired)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, marker="s", s=50, edgecolors='w', alpha=0.7, cmap=plt.cm.Paired)
plt.xlabel('Petal length')
plt.ylabel('Petal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.show()

print("TRAINING:")
y_hats_tr = reg.predict(X_train)
print("Acc:", metrics.accuracy_score(y_train, y_hats_tr))
# print("p/r/F/supp:",sklearn.metrics.precision_recall_fscore_support(y_train, y_hats_tr, average='micro'))
print ("Classification Report:")
print (metrics.classification_report(y_train, y_hats_tr))
print("\nConfusion Mx (training data):")
print(metrics.confusion_matrix(y_train, y_hats_tr))

print("\nTEST")
y_hats = reg.predict(X_test)
print("Acc:", metrics.accuracy_score(y_test, y_hats))
# print("p/r/F/supp:",sklearn.metrics.precision_recall_fscore_support(y_test, y_hats, average='micro'))
print ("Classification Report:")
print (metrics.classification_report(y_test, y_hats))
print("\nConfusion Mx (test data):")
print(metrics.confusion_matrix(y_test, y_hats))


In [ ]:
#now re-train with all the available flower features
X_train = X[train_indices, :]
X_test = X[test_indices, :]
print(X_train[0:10])

reg_allf = LogisticRegression()
reg_allf.fit(X_train, y_train)

y_hats = reg_allf.predict(X_test)

print(metrics.accuracy_score(y_test, y_hats))
print ("Classification Report:")
print (metrics.classification_report(y_test, y_hats))
print("\nConfusion Mx (test data):")
print(metrics.confusion_matrix(y_test, y_hats))


### Multinomial classification
Now reintroduce the setosa class, and try running the same routines again with a non-binary dataset.

## Summary
- We loaded the Iris dataset and manually split it into training and test sets
- We used Logistic Regression to classify the test data
- We checked out the decision boundary by plotting the class partitions
- We got P/R/F and Confusion Matrices on train and test
- We repeated with all 3 classes